In [1]:
# Standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom modules
from jpm_time_conversions import *
from jpm_logger import JpmLogger
from closest import closest
from jpm_number_printing import latex_float

__author__ = 'James Paul Mason'
__contact__ = 'jmason86@gmail.com'


def generate_jedi_catalog(threshold_time_prior_flare_minutes=240.0, 
                          dimming_window_relative_to_flare_minutes_left=0.0, 
                          dimming_window_relative_to_flare_minutes_right=240.0,
                          output_path='./', verbose=False):
    """Wrapper code for creating James's Extreme Ultraviolet Variability Experiment (EVE) Dimming Index (JEDI) catalog.

    Inputs:
        None.
        
    Optional Inputs:
        threshold_time_prior_flare_minutes [float]:             How long before a particular event does the last one need to have
                                                                occurred to be considered independent. If the previous one was too
                                                                recent, will use that event's pre-flare irradiance.
                                                                Default is 240 (4 hours).
        dimming_window_relative_to_flare_minutes_left [float]:  Defines the left side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. Negative numbers mean 
                                                                minutes prior to the flare peak. Default is 0.0.
        dimming_window_relative_to_flare_minutes_right [float]: Defines the right side of the time window to search for dimming
                                                                relative to the GOES/XRS flare peak. If another flare
                                                                occurs before this, that time will define the end of the
                                                                window instead. Default is 240 (4 hours).
        output_path [str]:                                      Set to a path for saving the JEDI catalog table and processing 
                                                                summary plots. Default is './', the current directory.
        verbose [bool]:                                         Set to log the processing messages to disk and console. Default is False.

    Outputs:
        No direct return, but writes a (csv? sql table? hdf5?) to disk with the dimming paramerization results.
        Subroutines also optionally save processing plots to disk in output_path.
                                                 
    Optional Outputs:
        None

    Example:
        generate_jedi_catalog(output_path='/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/',
                              verbose=True)
    """
    return 1

In [10]:
# prototype only: set up example inputs
%matplotlib inline
plt.style.use('jpm-transparent-light')
example_to_use = 'rising'

if example_to_use == 'oscillating':
    estimated_time_of_peak_start = pd.Timestamp('2012-04-15 21:20:00.0')  # oscillating light curve
elif example_to_use == 'rising':
    estimated_time_of_peak_start = pd.Timestamp('2011-02-16 14:20:00.0')  # rising light curve
else:
    estimated_time_of_peak_start = pd.Timestamp('2011-02-16 15:20:00.0')  # with prior flare

max_median_diff_threshold = 10.0
std_threshold = 0.5
plot_path_filename = './bla.png'
verbose = True

from scipy.io.idl import readsav
if example_to_use == 'oscillating':
    idl_generated_data = readsav('/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/Example Dimming Light Curve.sav')
    irradiance = idl_generated_data.irradiance[:, 3].byteswap().newbyteorder() # Not sure why this is big endian but pandas plotting needs it to be little
    light_curve_df = pd.DataFrame({'irradiance_percent': irradiance})
    light_curve_df['irradiance'] = light_curve_df['irradiance_percent'] * 6.2e-5 + 6.2e-5 # It's already in % so converting back
    light_curve_df.index = yyyydoy_sod_to_datetime(idl_generated_data.yyyydoy, idl_generated_data.sod) # Convert EVE to datetime
elif example_to_use == 'rising':
    idl_generated_data = readsav('/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/Example Pre-flare Light Curve 1.sav')
    irradiance = idl_generated_data.irradiance.byteswap().newbyteorder() # Not sure why this is big endian but pandas plotting needs it to be little
    light_curve_df = pd.DataFrame({'irradiance': irradiance, 
                                   'irradiance_percent': (irradiance - irradiance[0]) / irradiance[0] * 100.})
    light_curve_df.index = pd.to_datetime(idl_generated_data.time.astype(str))
else:
    idl_generated_data = readsav('/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/Example Pre-flare Light Curve 2.sav')
    irradiance = idl_generated_data.irradiance.byteswap().newbyteorder() # Not sure why this is big endian but pandas plotting needs it to be little
    light_curve_df = pd.DataFrame({'irradiance': irradiance, 
                                   'irradiance_percent': (irradiance - irradiance[0]) / irradiance[0] * 100.})
    light_curve_df.index = pd.to_datetime(idl_generated_data.time.astype(str))

light_curve_df.head()

,irradiance,irradiance_percent
2011-02-16 12:00:13,0.000061,0.000000
2011-02-16 12:01:13,0.000062,0.407284
2011-02-16 12:02:13,0.000062,0.580749
2011-02-16 12:03:13,0.000062,0.392872
2011-02-16 12:04:13,0.000062,0.419690


In [11]:
# Prepare the logger for verbose
if verbose:
    # TODO: Update the path
    logger = JpmLogger(filename='determine_preflare_irradiance_log', path='/Users/jmason86/Desktop/')
    logger.info("Running on event with peak start time of {0}.".format(estimated_time_of_peak_start))

2017-12-28 12:54:56,246 [INFO ] [<module>] Running on event with peak start time of 2011-02-16 14:20:00.
2017-12-28 12:54:56,246 [INFO ] [<module>] Running on event with peak start time of 2011-02-16 14:20:00.


In [1]:
# Save JEDI catalog